In [2]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [3]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [4]:
from monoHbbtools.Utilities import lumicalc

In [5]:
lumicalc.Generate_Luminosity_file("Zjetsnunu_MET_Run2018.coffea")

{316928: CompletedProcess(args='brilcalc -lumi -c web -r 316928', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 316944: CompletedProcess(args='brilcalc -lumi -c web -r 316944', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 315420: CompletedProcess(args='brilcalc -lumi -c web -r 315420', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 316455: CompletedProcess(args='brilcalc -lumi -c web -r 316455', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 316456: CompletedProcess(args='brilcalc -lumi -c web -r 316456', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 316457: CompletedProcess(args='brilcalc -lumi -c web -r 316457', returncode=127, stdout=b'', stderr=b'/bin/bash: line 1: brilcalc: command not found\n'),
 316469: CompletedProcess(args='brilcalc -lumi -c web -r 316469', retu

In [6]:
import subprocess

In [9]:
output = subprocess.run("ls", shell=True, executable="/bin/bash", capture_output=True)
print(output.stdout)

b'BackgroundDijets.coffea\nplotter_Zjetsnunu.py\nprocessor_Zjetsnunu.py\n__pycache__\nrunner_Zjetsnunu.py\nTest.ipynb\ntest_processor.py\ntest_runner.py\nZjetsnunu_MET_Run2018.coffea\nZjetsnunu_ZJets_NuNu.coffea\nZnunuCombined.png\nZnunuCombinedwithMETcut.png\nZnunuMET_Run2018.png\nZnunuZJets_NuNu.png\n'


In [20]:
out = subprocess.check_output(["ls","-lhrt"]).decode("ASCII")
out_list = out.split("\n")
with open("test.txt","w") as outfile :    
    outfile.writelines([line+"\n" for line in out_list])